# 주차수요 예측 EDA & Catboost Baseline
- 작성자 : 파베르
- 작성일자 : 2021.06.15
- 출처 : [링크](https://dacon.io/competitions/official/235745/codeshare/2851?page=1&dtype=recent)

In [1]:
import pandas as pd
import numpy as np

# 한글폰트 설정, 그래프 마이너스 표시 설정
import matplotlib
from matplotlib import font_manager, rc
from matplotlib import pyplot as plt
import platform
import seaborn as sns

if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

## 1. 데이터 탐색
### 1.1. 데이터 불러오기 & 간단한 탐색

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
age_gender_info = pd.read_csv('data/age_gender_info.csv')

- train 데이터에서는 임대보증금, 임대료, 도보 10분거리 내 지하철역 수(환승노선 수 반영), 도보 10분거리 내 버스정류장 수에 NULL값 존재
- test 데이터에서는 자격유형, 임대보증금, 임대료, 도보 10분거리 내 지하철역 수(환승노선 수 반영)에 NULL값 존재

In [3]:
# 임대보증금과 임대료는 '-'을 null로 바꾼 후 float로 타입 변경
train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
test['임대보증금'] = test['임대보증금'].astype(float)

train.loc[train.임대료=='-', '임대료'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
train['임대료'] = train['임대료'].astype(float)
test['임대료'] = test['임대료'].astype(float)

### 1.2. NULL 값이 있는 변수 탐색
#### (1) 임대보증금, 임대료

In [4]:
train[train.임대보증금.isnull()].공급유형.value_counts()

임대상가    562
국민임대      8
공공분양      7
행복주택      4
Name: 공급유형, dtype: int64

In [5]:
test[test.임대보증금.isnull()].공급유형.value_counts()

임대상가    177
영구임대      5
행복주택      4
Name: 공급유형, dtype: int64

In [6]:
train[train.임대보증금.isnull()].자격유형.value_counts()

D    569
H      8
K      4
Name: 자격유형, dtype: int64

In [7]:
test[test.임대보증금.isnull()].자격유형.value_counts()

D    180
L      4
C      2
Name: 자격유형, dtype: int64

In [8]:
train[train.자격유형.isin(['D', 'H', 'K'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 757 entries, 88 to 2919
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          757 non-null    object 
 1   총세대수                          757 non-null    int64  
 2   임대건물구분                        757 non-null    object 
 3   지역                            757 non-null    object 
 4   공급유형                          757 non-null    object 
 5   전용면적                          757 non-null    float64
 6   전용면적별세대수                      757 non-null    int64  
 7   공가수                           757 non-null    float64
 8   자격유형                          757 non-null    object 
 9   임대보증금                         176 non-null    float64
 10  임대료                           176 non-null    float64
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  623 non-null    float64
 12  도보 10분거리 내 버스정류장 수            757 non-null    float64
 13  단지내

In [9]:
test[test.자격유형.isin(['D', 'L', 'C'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226 entries, 73 to 1017
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          226 non-null    object 
 1   총세대수                          226 non-null    int64  
 2   임대건물구분                        226 non-null    object 
 3   지역                            226 non-null    object 
 4   공급유형                          226 non-null    object 
 5   전용면적                          226 non-null    float64
 6   전용면적별세대수                      226 non-null    int64  
 7   공가수                           226 non-null    float64
 8   자격유형                          226 non-null    object 
 9   임대보증금                         40 non-null     float64
 10  임대료                           40 non-null     float64
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  201 non-null    float64
 12  도보 10분거리 내 버스정류장 수            226 non-null    float64
 13  단지내

In [10]:
# 임대보증금과 임대료가 NULL인 경우는 0으로 대체하는 것이 좋아보임
train[['임대보증금', '임대료']] = train[['임대보증금', '임대료']].fillna(0)
test[['임대보증금', '임대료']] = test[['임대보증금', '임대료']].fillna(0)

#### (2) 도보 10분거리 내 지하철역 수(환승노선 수 반영), 도보 10분거리 내 버스정류장 수
- NULL 값을 0으로 대체

In [11]:
subway_null_codes = train[train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()].단지코드.unique()
train.loc[train.단지코드.isin(subway_null_codes), '도보 10분거리 내 지하철역 수(환승노선 수 반영)'].value_counts(dropna=False)


NaN    211
Name: 도보 10분거리 내 지하철역 수(환승노선 수 반영), dtype: int64

In [12]:
subway_null_codes = test[test['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()].단지코드.unique()
test.loc[test.단지코드.isin(subway_null_codes), '도보 10분거리 내 지하철역 수(환승노선 수 반영)'].value_counts(dropna=False)

NaN    42
Name: 도보 10분거리 내 지하철역 수(환승노선 수 반영), dtype: int64

In [13]:
bus_null_codes = train[train['도보 10분거리 내 버스정류장 수'].isnull()].단지코드.unique()
train.loc[train.단지코드.isin(bus_null_codes), '도보 10분거리 내 버스정류장 수'].value_counts(dropna=False)

NaN    4
Name: 도보 10분거리 내 버스정류장 수, dtype: int64

In [14]:
cols = ['도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수']
train[cols] = train[cols].fillna(0)
test[cols] = test[cols].fillna(0)

#### (3) 자격유형

In [15]:
test[test.자격유형.isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000.0,37470.0,0.0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000.0,44770.0,0.0,2.0,173.0


In [16]:
test[test.단지코드=='C2411'] # A로 채우면 될듯

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
193,C2411,962,아파트,경상남도,국민임대,39.43,56,25.0,A,11992000.0,100720.0,0.0,2.0,840.0
194,C2411,962,아파트,경상남도,국민임대,39.72,336,25.0,A,11992000.0,100720.0,0.0,2.0,840.0
195,C2411,962,아파트,경상남도,국민임대,39.82,179,25.0,A,11992000.0,100720.0,0.0,2.0,840.0
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000.0,37470.0,0.0,2.0,840.0
197,C2411,962,아파트,경상남도,국민임대,51.93,150,25.0,A,21586000.0,171480.0,0.0,2.0,840.0


In [17]:
test.loc[test.단지코드.isin(['C2411']) & test.자격유형.isnull(), '자격유형'] = 'A'

In [18]:
test[test.단지코드=='C2253'].head() # 임대보증금과 임대료가 존재하는 경우 자격유형이 C => C로 채우면 될듯

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000.0,44770.0,0.0,2.0,173.0
259,C2253,1161,아파트,강원도,영구임대,31.32,239,0.0,C,3731000.0,83020.0,0.0,2.0,173.0
260,C2253,1161,아파트,강원도,영구임대,31.32,149,0.0,C,3731000.0,83020.0,0.0,2.0,173.0
261,C2253,1161,상가,강원도,임대상가,13.77,1,0.0,D,0.0,0.0,0.0,2.0,173.0
262,C2253,1161,상가,강원도,임대상가,22.89,1,0.0,D,0.0,0.0,0.0,2.0,173.0


In [19]:
test.loc[test.단지코드.isin(['C2253']) & test.자격유형.isnull(), '자격유형'] = 'C'

### 1.3. 중복확인

In [20]:
train.shape, train.drop_duplicates().shape # 중복 존재

((2952, 15), (2632, 15))

In [21]:
test.shape, test.drop_duplicates().shape # 중복 존재

((1022, 14), (949, 14))

In [22]:
# 중복되는 데이터는 제거하고 분석을 진행하는 것이 좋아보임
train = train.drop_duplicates()
test = test.drop_duplicates()

In [23]:
train[train.단지코드=='C2483']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000.0,103680.0,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000.0,103680.0,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
5,C2483,900,아파트,경상북도,국민임대,51.95,89,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
6,C2483,900,아파트,경상북도,국민임대,51.95,135,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
7,C2483,900,아파트,경상북도,국민임대,59.88,86,38.0,A,30357000.0,214270.0,0.0,3.0,1425.0,1015.0


In [24]:
print(f"단지코드 C2483에서 유일한 값을 가지는 변수들:\n{list(train.columns[train[train.단지코드=='C2483'].nunique()==1])}")


단지코드 C2483에서 유일한 값을 가지는 변수들:
['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '공가수', '자격유형', '도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수']


In [25]:
train.groupby(['단지코드']).nunique(dropna=False)

,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
단지코드,,,,,,,,,,,,,,
C1000,1,1,1,1,5,6,1,1,3,3,1,1,1,1
C1004,1,2,1,2,15,3,1,2,3,3,1,1,1,1
C1005,1,1,1,1,3,3,1,1,3,3,1,1,1,1
C1013,1,1,1,1,4,5,1,1,3,3,1,1,1,1
C1014,1,1,1,1,6,7,1,1,4,4,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C2663,1,1,1,1,5,6,1,1,4,4,1,1,1,1
C2666,1,1,1,1,4,6,1,1,2,2,1,1,1,1
C2670,1,1,1,1,4,3,1,1,3,3,1,1,1,1


In [26]:
train.groupby(['단지코드']).nunique(dropna=False).sum(axis=0)

총세대수                             423
임대건물구분                           456
지역                               423
공급유형                             488
전용면적                            1898
전용면적별세대수                        2230
공가수                              423
자격유형                             510
임대보증금                           1277
임대료                             1289
도보 10분거리 내 지하철역 수(환승노선 수 반영)     423
도보 10분거리 내 버스정류장 수               423
단지내주차면수                          423
등록차량수                            423
dtype: int64

- 값이 423보다 크면 하나의 단지코드에 대해 둘 이상의 항목이 존재하는 것
- 임대건물구분, 공급유형, 전용면적, 전용면적별세대수, 자격유형, 임대보증금, 임대료는 하나의 단지코드에 대해 둘 이상의 항목 존재
- 단지코드별 집계시 총세대수, 지역, 공가수, 도보 10분거리 내 지하철역 수(환승노선 수 반영), 도보 10분거리 내 버스정류장 수, 단지내주차면수, 등록차량수는 그대로 사용하고, 나머지 변수들은 각 항목들을 변수로 만들어 사용하는 것이 좋아보임

### 1.4. 단지코드별 집계
#### (1)  하나의 단지코드에 하나의 값만 존재하는 변수들
- 총세대수, 지역, 공가수, 도보 10분거리 내 지하철역 수(환승노선 수 반영), 도보 10분거리 내 버스정류장 수, 단지내주차면수, 등록차량수

In [27]:
unique_cols = ['총세대수', '지역', '공가수', 
               '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
               '도보 10분거리 내 버스정류장 수',
               '단지내주차면수', '등록차량수']
train_agg = train.set_index('단지코드')[unique_cols].drop_duplicates()
test_agg = test.set_index('단지코드')[[col for col in unique_cols if col!='등록차량수']].drop_duplicates()


In [28]:
train_agg

,총세대수,지역,공가수,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
단지코드,,,,,,,
C2483,900,경상북도,38.0,0.0,3.0,1425.0,1015.0
C2515,545,경상남도,17.0,0.0,3.0,624.0,205.0
C1407,1216,대전광역시,13.0,1.0,1.0,1285.0,1064.0
C1945,755,경기도,6.0,1.0,3.0,734.0,730.0
C1470,696,전라북도,14.0,0.0,2.0,645.0,553.0
...,...,...,...,...,...,...,...
C2586,90,제주특별자치도,7.0,0.0,3.0,66.0,57.0
C2035,492,강원도,24.0,0.0,1.0,521.0,246.0
C2020,40,부산광역시,7.0,1.0,2.0,25.0,19.0


In [29]:
test_agg

,총세대수,지역,공가수,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
단지코드,,,,,,
C1072,754,경기도,14.0,0.0,2.0,683.0
C1128,1354,경기도,9.0,0.0,3.0,1216.0
C1456,619,부산광역시,18.0,0.0,16.0,547.0
C1840,593,전라북도,7.0,0.0,3.0,543.0
C1332,1297,경기도,11.0,0.0,2.0,1112.0
...,...,...,...,...,...,...
C2456,349,제주특별자치도,17.0,0.0,4.0,270.0
C1266,596,충청북도,35.0,0.0,1.0,593.0
C2152,120,강원도,9.0,0.0,1.0,40.0


#### (2) 하나의 단지코드에 둘 이상의 값이 존재하는 변수들
- 임대건물구분, 공급유형, 전용면적, 전용면적별세대수, 자격유형, 임대보증금, 임대료

In [30]:
def reshape_cat_features(data, cast_col, value_col):
    res = data.drop_duplicates(['단지코드', cast_col]).assign(counter=1).pivot(index='단지코드', columns=cast_col, values=value_col).fillna(0)
    res.columns.name = None
    res = res.rename(columns={col:cast_col+'_'+col for col in res.columns})
    return res

#### 1) 임대건물구분: 아파트/상가
- 주상복합인 경우 주차수요가 주거only인 경우보다 많지 않을까? => 상가 직원들도 차량 등록이 가능하면 상가 당 주차수요는 일반 가구보다 높을 수 있음
- 상가의 전용면적별세대수는 모두 1
- 우선 임대건물구분을 열로 올려서 단지가 주상복합인지 아닌지 구분할 수 있게 해야함

In [34]:
#  train.loc[train.임대건물구분 == '상가','전용면적별세대수']

88     1
91     1
101    1
102    1
104    1
      ..
811    1
817    1
821    1
826    1
838    1
Name: 전용면적별세대수, Length: 285, dtype: int64

In [35]:
reshape_cat_features(data=train, cast_col='임대건물구분', value_col='counter')

,임대건물구분_상가,임대건물구분_아파트
단지코드,,
C1000,0.0,1.0
C1004,1.0,1.0
C1005,0.0,1.0
C1013,0.0,1.0
C1014,0.0,1.0
...,...,...
C2663,0.0,1.0
C2666,0.0,1.0
C2670,0.0,1.0


In [36]:
reshape_cat_features(data=test, cast_col='임대건물구분', value_col='counter')

,임대건물구분_상가,임대건물구분_아파트
단지코드,,
C1003,0.0,1.0
C1006,1.0,1.0
C1016,0.0,1.0
C1019,0.0,1.0
C1030,0.0,1.0
...,...,...
C2653,0.0,1.0
C2675,0.0,1.0
C2676,1.0,1.0


#### 2) 공급유형
- [국민임대와 공공임대의 차이](https://brunch.co.kr/@leeeeesh/91)
- LH공사와 SH공사에서 제공하는 __국민임대__는 국가 재정과 국민주택기금을 지원받아 국가, 지방자치단체, 한국토지주택공사 또는 지방공사가 건설, 공급하는 주택을 의미한다. 저렴한 임대료를 지불하고 장기(30년) 임대가 가능하다. 장기 거주는 가능하지만 해당 주거지를 매입할 수 없다는 단점이 있다.
- 반면 __공공임대__는 5년 또는 10년 후 임대기간이 종료되었을 때 입주자에게 우선 분양전환혜택을 준다.

In [39]:
# train.공급유형.unique()

array(['국민임대', '공공임대(50년)', '영구임대', '임대상가', '공공임대(10년)', '공공임대(분납)',
       '장기전세', '공공분양', '행복주택', '공공임대(5년)'], dtype=object)

In [40]:
pd.concat([train.공급유형.value_counts(), test.공급유형.value_counts()], axis=1)

,공급유형,공급유형
국민임대,1730,619.0
임대상가,285,112.0
공공임대(10년),203,34.0
행복주택,203,121.0
영구임대,149,44.0
공공임대(50년),31,13.0
공공임대(분납),12,6.0
장기전세,9,NaN
공공분양,7,NaN
공공임대(5년),3,NaN


In [42]:
train.loc[train.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
test.loc[test.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
train.loc[train.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'
test.loc[test.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'

In [43]:
set(train.공급유형)

{'공공임대(50년)', '공공임대(5년/10년/분납/분양)', '국민임대/장기전세', '영구임대', '임대상가', '행복주택'}

In [44]:
set(test.공급유형)

{'공공임대(50년)', '공공임대(5년/10년/분납/분양)', '국민임대/장기전세', '영구임대', '임대상가', '행복주택'}

In [45]:
reshape_cat_features(data=train, cast_col='공급유형', value_col='counter')

,공급유형_공공임대(50년),공급유형_공공임대(5년/10년/분납/분양),공급유형_국민임대/장기전세,공급유형_영구임대,공급유형_임대상가,공급유형_행복주택
단지코드,,,,,,
C1000,0.0,0.0,1.0,0.0,0.0,0.0
C1004,0.0,0.0,0.0,1.0,1.0,0.0
C1005,0.0,0.0,1.0,0.0,0.0,0.0
C1013,0.0,0.0,1.0,0.0,0.0,0.0
C1014,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
C2663,0.0,0.0,1.0,0.0,0.0,0.0
C2666,0.0,0.0,1.0,0.0,0.0,0.0
C2670,0.0,0.0,1.0,0.0,0.0,0.0


In [46]:
reshape_cat_features(data=test, cast_col='공급유형', value_col='counter')

,공급유형_공공임대(50년),공급유형_공공임대(5년/10년/분납/분양),공급유형_국민임대/장기전세,공급유형_영구임대,공급유형_임대상가,공급유형_행복주택
단지코드,,,,,,
C1003,0.0,0.0,0.0,0.0,0.0,1.0
C1006,0.0,0.0,0.0,1.0,1.0,0.0
C1016,0.0,0.0,1.0,0.0,0.0,0.0
C1019,0.0,0.0,1.0,0.0,0.0,0.0
C1030,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
C2653,0.0,0.0,1.0,0.0,0.0,0.0
C2675,0.0,0.0,1.0,0.0,0.0,0.0
C2676,0.0,0.0,0.0,1.0,1.0,0.0


#### 3) 자격유형
- 변수생성 컨셉: 특정자격 유형이 많으면 그 단지는 등록차량수가 적을 것이다.
- 예를들어, 소득수준이 낮은 자격유형의 세대가 많으면 주차수요가 적을 가능성이 높을 것으로 예상해볼 수 있다.
- 문제는 특성이 비슷한 항목끼리 묶어야 의미가 있을 것 같은데, 비식별화 되어 있어서 각 코드값이 무엇인지 정확히 알 수 없다는 것임
- 단지코드별로 자격유형별 세대수를 알 수 있으면 좋을 것 같지만 이 데이터에서는 파악 불가능
- 자격유형별 소득수준을 간접적으로 파악하기 위해 자격유형별 임대보증금의 평균, 임대료의 평균을 변수로 사용해 볼 수 있을 것 같음

In [47]:
pd.concat([train.자격유형.value_counts(), 
           test.자격유형.value_counts()], axis=1)

,자격유형,자격유형
A,1775,569.0
D,292,114.0
H,154,92.0
J,105,81.0
C,92,35.0
I,49,7.0
E,37,10.0
K,33,16.0
L,33,12.0
N,29,10.0


- B, F, O가 train에만 있고 test에는 없음

- [공공임대주택 공급대상](https://www.myhome.go.kr/hws/portal/cont/selectContRentalView.do#guide=RH101)

- 영구임대: 생계급여 또는 의료급여 수급자 등[소득 1분위] - 1
- 국민임대: 무주택세대구성원[소득 2~4분위] - 3
- 장기전세: 무주택세대구성원[소득 3~4분위] - 2
- 공공임대(5년/10년/분납): 무주택세대구성원[소득 3~5분위] - 3
- 행복주택: 무주택세대구성원/무주택자[소득 2~5분위] - 4
- 이렇게 나누면 13개 항목인데 자격유형의 항목수는 15개

In [48]:
train.loc[train.자격유형=='B', ['임대건물구분', '공급유형']].drop_duplicates()
# 자격유형 B는 공급유형이 모두 국민임대/장기전세, 공급유형이 국민임대/장기전세 일때 자격유형이 어떤게 있는지 보는게 좋을 듯


,임대건물구분,공급유형
26,아파트,국민임대/장기전세


In [49]:
train.loc[train.공급유형.isin(['국민임대/장기전세']), '자격유형'].value_counts()

A    1524
H     154
E      34
B      18
G       9
Name: 자격유형, dtype: int64

국민임대/장기전세

- (일반)해당지역 거주 무주택세대구성원
- (특별/우선)3자녀 이상 가구
- (특별/우선)국가유공자
- (특별/우선)영구임대 입주자
- (특별/우선)비닐간이공작물 거주자
- (특별/우선)신혼부부(혼인기간 5년이내)
- (특별/우선)사업지구 철거민
- (특별/우선)기타 공급대상(고령자, 노부모부양자, 장애인, 파독근로자 등)
- 건수가 가장 많은 A는 해당지역 거주 무주택세대구성원으로 예상됨


In [50]:
# 다른 공급유형별 자격유형도 살펴보자
train.loc[train.공급유형.isin(['영구임대']), '자격유형'].value_counts()

C    92
I    49
E     3
F     3
A     2
Name: 자격유형, dtype: int64

영구임대

- (일반)생계급여 또는 의료급여 수급자
- (일반)국가유공자
- (특별/우선)수급자 선정기준의 소득인정액 이하인 국가유공자
- (특별/우선)귀환국군포로
- (특별/우선)수급자 신혼부부

In [51]:
train.loc[train.공급유형.isin(['공공임대(5년/10년/분납/분양)']), '자격유형'].value_counts()


A    218
D      7
Name: 자격유형, dtype: int64

공공임대(5년/10년/분납/분양)

- (일반)해당지역 거주무주택세대구성원
- (특별/우선)다자녀 특별
- (특별/우선)신혼부부 특별
- (특별/우선)생애최초 특별
- (특별/우선)노부모부양 특별
- (특별/우선)국가유공자 특별
- (특별/우선)기타 특별(장애인, 철거민 등)
- A는 '(일반)해당지역 거주무주택세대구성원'으로 추정됨

In [52]:
train.loc[train.공급유형.isin(['행복주택']), '자격유형'].value_counts()

J    105
K     33
L     33
N     29
M      2
O      1
Name: 자격유형, dtype: int64

행복주택

- 대학생(취준생 포함), 사회초년생(제취준생 포함), 신혼부부(예비신혼부부, 대학생, 취준생 신혼부부 포함), 고령자, 주거급여수급자, 산업단지 근로자
- 이렇게 6개 항목이랑 매핑 될듯

In [54]:
train.loc[train.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'
test.loc[test.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'

In [55]:
train.loc[train.자격유형.isin(['A']), '공급유형'].value_counts()

국민임대/장기전세             1524
공공임대(5년/10년/분납/분양)     218
공공임대(50년)               31
영구임대                     2
Name: 공급유형, dtype: int64

In [56]:
train.loc[train.자격유형.isin(['D']), '공급유형'].value_counts()

임대상가                  285
공공임대(5년/10년/분납/분양)      7
Name: 공급유형, dtype: int64

In [57]:
train.loc[train.자격유형.isin(['H']), '공급유형'].value_counts()

국민임대/장기전세    154
Name: 공급유형, dtype: int64

In [58]:
train.loc[train.자격유형.isin(['C']), '공급유형'].value_counts()

영구임대    92
Name: 공급유형, dtype: int64

In [59]:
train.loc[train.자격유형.isin(['I']), '공급유형'].value_counts()

영구임대    49
Name: 공급유형, dtype: int64

In [60]:
train.loc[train.자격유형.isin(['E']), '공급유형'].value_counts()

국민임대/장기전세    34
영구임대          3
Name: 공급유형, dtype: int64

In [63]:
train.loc[train.자격유형.isin(['B']), '공급유형'].value_counts()

국민임대/장기전세    18
Name: 공급유형, dtype: int64

In [64]:
train.loc[train.자격유형.isin(['G']), '공급유형'].value_counts()

국민임대/장기전세    9
Name: 공급유형, dtype: int64

In [65]:
train.loc[train.자격유형.isin(['F']), '공급유형'].value_counts()

영구임대    3
Name: 공급유형, dtype: int64

- 'B','G','H', 'E'는 국민임대/장기전세 공급대상 (E는 영구임대인 경우도 있긴 하지만 국민임대/장기전세인 경우가 많아서 여기에 포함)
- 'C', 'I', 'F' 는 영구임대 공급대상으로 묶어 보겠음

In [67]:
train.loc[train.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대/장기전세_공급대상'
test.loc[test.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대/장기전세_공급대상'

train.loc[train.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'
test.loc[test.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'

In [68]:
pd.concat([train.자격유형.value_counts(), 
           test.자격유형.value_counts()], axis=1)

,자격유형,자격유형
A,1775,569
D,292,114
국민임대/장기전세_공급대상,218,103
행복주택_공급대상,203,121
영구임대_공급대상,144,42


In [69]:
train.공급유형.unique()

array(['국민임대/장기전세', '공공임대(50년)', '영구임대', '임대상가', '공공임대(5년/10년/분납/분양)',
       '행복주택'], dtype=object)

In [70]:
reshape_cat_features(data=train, cast_col='자격유형', value_col='counter')

,자격유형_A,자격유형_D,자격유형_국민임대/장기전세_공급대상,자격유형_영구임대_공급대상,자격유형_행복주택_공급대상
단지코드,,,,,
C1000,1.0,0.0,0.0,0.0,0.0
C1004,0.0,1.0,0.0,1.0,0.0
C1005,1.0,0.0,0.0,0.0,0.0
C1013,1.0,0.0,0.0,0.0,0.0
C1014,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
C2663,0.0,0.0,1.0,0.0,0.0
C2666,1.0,0.0,0.0,0.0,0.0
C2670,1.0,0.0,0.0,0.0,0.0


In [71]:
reshape_cat_features(data=test, cast_col='자격유형', value_col='counter')

,자격유형_A,자격유형_D,자격유형_국민임대/장기전세_공급대상,자격유형_영구임대_공급대상,자격유형_행복주택_공급대상
단지코드,,,,,
C1003,0.0,0.0,0.0,0.0,1.0
C1006,0.0,1.0,0.0,1.0,0.0
C1016,1.0,0.0,0.0,0.0,0.0
C1019,1.0,0.0,0.0,0.0,0.0
C1030,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...
C2653,1.0,0.0,0.0,0.0,0.0
C2675,1.0,0.0,0.0,0.0,0.0
C2676,0.0,1.0,0.0,1.0,0.0


In [72]:
train_agg = pd.concat([train_agg,
                       reshape_cat_features(data=train, cast_col='임대건물구분', value_col='counter'),
                       reshape_cat_features(data=train, cast_col='공급유형', value_col='counter'),
                       reshape_cat_features(data=train, cast_col='자격유형', value_col='counter')], axis=1)

test_agg = pd.concat([test_agg,
                       reshape_cat_features(data=test, cast_col='임대건물구분', value_col='counter'),
                       reshape_cat_features(data=test, cast_col='공급유형', value_col='counter'),
                       reshape_cat_features(data=test, cast_col='자격유형', value_col='counter')], axis=1)

In [73]:
train_agg.shape, test_agg.shape

((423, 20), (150, 19))

## 2. Catboost 모델링

In [74]:
cat_features = ['지역']
target_col = '등록차량수'

In [75]:
from catboost import CatBoostRegressor
catb = CatBoostRegressor(
         cat_features=cat_features,
         loss_function='MAE',
         n_estimators=500, 
         learning_rate=0.05, 
         random_state=42
    )
    
catb.fit(train_agg.drop(columns=[target_col]), train_agg[target_col], verbose=100)

0:	learn: 279.3964528	total: 64.4ms	remaining: 32.1s
100:	learn: 100.3025011	total: 203ms	remaining: 801ms
200:	learn: 75.2070254	total: 346ms	remaining: 515ms
300:	learn: 62.5859656	total: 489ms	remaining: 323ms
400:	learn: 54.0292543	total: 644ms	remaining: 159ms
499:	learn: 50.0553217	total: 790ms	remaining: 0us


In [76]:
sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission.head()

,code,num
0,C1072,0
1,C1128,0
2,C1456,0
3,C1840,0
4,C1332,0


In [77]:
sample_submission['num'] = catb.predict(test_agg)

In [78]:
sample_submission

,code,num
0,C1072,678.748011
1,C1128,1183.305560
2,C1456,568.547066
3,C1840,519.937576
4,C1332,1050.366626
...,...,...
145,C2456,262.919072
146,C1266,428.923075
147,C2152,50.028510
148,C1267,285.955245


In [79]:
sample_submission.to_csv('submission_catb_baseline_20210622.csv', index=False)